In [ ]:
import os
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
import chromadb
from dotenv import load_dotenv
from sentence_transformers import InputExample
from sentence_transformers import SentenceTransformer, losses
from torch.utils.data import DataLoader


ModuleNotFoundError: No module named 'chromadb'

In [ ]:
import torch

In [ ]:
!pip uninstall -y tensorflow --quiet
!pip install git+https://github.com/ludwig-ai/ludwig.git@master --quiet
!pip install "git+https://github.com/ludwig-ai/ludwig.git@master#egg=ludwig[llm]" --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.6/39.6 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 14.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.0/232.0 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 57.5 MB/s eta 0:00

In [ ]:
from IPython.display import HTML, display
import torch

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)

def clear_cache():
  if torch.cuda.is_available():
    model = None
    torch.cuda.empty_cache()

In [ ]:
import getpass
import locale; locale.getpreferredencoding = lambda: "UTF-8"
import logging
import os
import pandas as pd
import torch
import yaml

from ludwig.api import LudwigModel


os.environ["HUGGING_FACE_HUB_TOKEN"] = getpass.getpass("Token:")
assert os.environ["HUGGING_FACE_HUB_TOKEN"]

Token:··········


In [ ]:
load_dotenv()

True

In [ ]:
def get_data(database_url):
    engine = create_engine(database_url)
    query = "SELECT * FROM reddit_usernames_comments"
    df = pd.read_sql(query, engine)
    return df

In [ ]:
database_url = os.getenv('DATABASE_URL')
df = get_data(database_url)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3276 entries, 0 to 3275
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   username  3276 non-null   object
 1   comments  3276 non-null   object
dtypes: object(2)
memory usage: 51.3+ KB


In [ ]:
df.head()

,username,comments
0,LoveAGoodTwist,"Female, Kentucky. 4 years out. Work equine on..."
1,wahznooski,"As a woman of reproductive age, fuck Texas|As ..."
2,Churro_The_fish_Girl,what makes you want to become a vet?|what make...
3,abarthch,"I see of course there are changing variables, ..."
4,VoodooKing,I have 412+ and faced issues because wireguard...


In [ ]:
df['comments'] = df['comments'].str.split('|').str[0] # Grab the first comment before |

In [ ]:
df['comments'] = df['comments'].str.replace(r'http\S+', '', regex=True)  # remove links
df['comments'] = df['comments'].str.replace(r'[^\x00-\x7F]+', '', regex=True)  # remove emojis

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3276 entries, 0 to 3275
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   username  3276 non-null   object
 1   comments  3276 non-null   object
dtypes: object(2)
memory usage: 51.3+ KB


In [ ]:
df['comments'][0]

'Female, Kentucky.  4 years out. Work equine only private practice. Base salary $85k plus bonuses/production which was $20k 2023. 6 days a week Jan-June/July then variable in the off season. No limit on PTO - took ~5 weeks last year. One paid conference a year (registration/travel/ 1/2 hotel/ transportation) or online CE program. All licensures & professional group fees covered. Cell phone allowance and mileage reimbursement.'

In [ ]:
# Check if username column has duplicates
df['username'].duplicated().any()

False

In [ ]:
chroma_client = chromadb.Client()

In [ ]:
def update_chromadb_collection(df):

    collection = chroma_client.get_or_create_collection(name="job_postings")


    # Prepare documents and metadatas for ChromaDB
    documents = list(df['comments'])

    ids = list(df['username'])

    # Update the ChromaDB collection
    collection.add(documents=documents, ids=ids)

In [ ]:
update_chromadb_collection(df)

Insert of existing embedding ID: LoveAGoodTwist
Insert of existing embedding ID: wahznooski
Insert of existing embedding ID: Churro_The_fish_Girl
Insert of existing embedding ID: abarthch
Insert of existing embedding ID: VoodooKing
Insert of existing embedding ID: queerofengland
Insert of existing embedding ID: tikitessie
Insert of existing embedding ID: theophania808
Insert of existing embedding ID: waterpencilboop
Insert of existing embedding ID: paxbanana0
Insert of existing embedding ID: Different-Ad1735
Insert of existing embedding ID: drawntage
Insert of existing embedding ID: Most-Exit-5507
Insert of existing embedding ID: Belikus
Insert of existing embedding ID: Silver_Desk2146
Insert of existing embedding ID: exPat17
Insert of existing embedding ID: ZQueen1
Insert of existing embedding ID: Fabulous_Cap_1146
Insert of existing embedding ID: 3_Black_Cats
Insert of existing embedding ID: PixelosPol
Insert of existing embedding ID: Frizzyawkward
Insert of existing embedding ID: Ca

In [ ]:
def query_collection(query):

    collection = chroma_client.get_collection(name="job_postings")

    # Query the collection with the candidate text and get the top 50 results
    results = collection.query(query_texts=query, n_results=len(df))

    ids = results['ids'][0]
    documents = results['documents'][0]
    distances = results['distances'][0]

    dataframe = pd.DataFrame({
    'username': ids,
    'comments': documents,
    'distances': distances
    })

    return dataframe

In [ ]:
doctor_query = ['a practicing medical doctor']
medical_doctor_df = query_collection(doctor_query)
medical_doctor_df[['distances', 'comments']].to_csv('meddoctors.csv', index=False)

In [ ]:
medical_doctor_df = medical_doctor_df.iloc[:91]


In [ ]:
medical_doctor_df['label'] = 'Medical Doctor'

In [ ]:
vet__doctor_query = ['a veterinary doctor', 'a vet doctor', 'a veterinarian']
vet_doctor_df = query_collection(vet__doctor_query)

In [ ]:
vet_doctor_df = vet_doctor_df.iloc[:93]

In [ ]:
vet_doctor_df['label'] = 'Veterinarian'

In [ ]:
med_student_query = ['a medical school student?', 'a vet student', 'a student']
med_student_df = query_collection(med_student_query)

In [ ]:
med_student = med_student_df.iloc[:25]

In [ ]:
nurse_query = ['a nurse', 'a therapist', 'a registered nurse']
nurse_df = query_collection(nurse_query)
nurse_df[['distances', 'comments']].to_csv('nurse.csv', index=False)

In [ ]:
nurse = nurse_df.iloc[:30]

In [ ]:
vet_tech_query = ['a vet tech', 'a vet tech student']
vet_tech_df = query_collection(vet_tech_query)
vet_tech_df[['distances', 'comments']].to_csv('vet_tech.csv', index=False)

In [ ]:
vet_tech = vet_tech_df.iloc[:33]

In [ ]:
others = pd.concat([vet_tech, nurse, med_student]).drop_duplicates(subset='username', keep='first')

In [ ]:
others['label'] = 'Other'

In [ ]:
combined_df = pd.concat([medical_doctor_df, vet_doctor_df, others]).drop_duplicates(subset='username', keep='first')

In [ ]:
# Map string labels to numerical labels
label_mapping = {'Medical Doctor': 0, 'Veterinarian': 1, 'Other': 2}
combined_df['label'] = combined_df['label'].map(label_mapping)

In [ ]:
train_df, val_df = train_test_split(combined_df, test_size=0.1, random_state=42)

In [ ]:
# Convert the training and validation dataframes into lists of InputExample
train_examples = [InputExample(texts=[text], label=label) for text, label in zip(train_df['comments'], train_df['label'])]
val_examples = [InputExample(texts=[text], label=label) for text, label in zip(val_df['comments'], val_df['label'])]

In [ ]:
# Load the pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')
tokenizer = model.tokenizer

/Users/honesty/Desktop/Sunday/all_works/challenge/ven/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    texts = [example.texts[0] for example in batch]  # Extract texts from InputExamples and ensure they are strings
    labels = [example.label for example in batch]  # Extract labels from InputExamples

    # Tokenize the texts
    tokenized_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

    # Convert labels to a tensor
    labels_tensor = torch.tensor(labels, dtype=torch.long)

    return tokenized_texts, labels_tensor

# Use the custom collate function in your DataLoader
train_dataloader = DataLoader(train_examples, batch_size=16, shuffle=True, collate_fn=collate_fn)

/Users/honesty/Desktop/Sunday/all_works/challenge/ven/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Define a DataLoader for the training examples
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

In [ ]:
# Define the loss function
train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=3)

In [3]:
# Fine-tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=4, warmup_steps=100)

In [2]:
# Map string labels to numerical labels
label_mapping = {'Medical Doctor': 0, 'Veterinarian': 1, 'Other': 2}
combined_df['label'] = combined_df['label'].map(label_mapping)

# Split the combined dataframe into training and validation sets
train_df, val_df = train_test_split(combined_df, test_size=0.1, random_state=42)

# Convert the training and validation dataframes into lists of InputExample
train_examples = [InputExample(texts=[text], label=label) for text, label in zip(train_df['comments'], train_df['label'])]
val_examples = [InputExample(texts=[text], label=label) for text, label in zip(val_df['comments'], val_df['label'])]

# Load the pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Define a DataLoader for the training examples
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

# Define the loss function
train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=3)

# Fine-tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=4, warmup_steps=100)

In [ ]:
instruction = "Imagine you're an expert in undertanding natural languages, the following is a comment made on a post on reddit, \
    your task is to analyze the comment and state who made the comment. You should only indicate Medical Doctor, Vetrinarian or Other. \
        Note: \n \
        Medical Doctor: This should only include practicing doctor, medical school students or nurses or medical professionals who aren’t doctors should go into be Other. \n \
        Veterinarian: This should only include practicing vet, vet students or vet techs should be regarded as Other \
        Other: All other persons apart from practicing medical doctor and veterinarian should be Other. "

In [ ]:
df = pd.read_csv('sample_data/data.csv')

In [ ]:
df['instruction'] = instruction

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.3)

In [ ]:
import yaml

config_str = """
model_type: llm
base_model: meta-llama/Llama-2-7b-hf

adapter:
  type: lora

quantization:
  bits: 4


prompt:
  template: |
    ### Instruction:
    {instruction}

    ### Input:
    {comments}

    ### Response:

input_features:
  - name: comments
    type: text
    preprocessing:
      max_sequence_length: 500

output_features:
  - name: label
    type: text
    preprocessing:
      max_sequence_length: 256

trainer:
  type: finetune
  learning_rate: 0.0001
  batch_size: 1
  gradient_accumulation_steps: 16
  epochs: 3
  learning_rate_scheduler:
    warmup_fraction: 0.01

preprocessing:
  sample_ratio: 0.1
"""

config = yaml.safe_load(config_str)

In [1]:
model = LudwigModel(config=config, logging_level=logging.INFO)
results = model.train(dataset=train_df)